---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    import re
    import pandas as pd
    import numpy as np
    import math
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    university_town = pd.read_csv('university_towns.txt', sep='\n', header=None, dtype=str)
    university_town.columns = ['RegionName']

    university_town['State'] = university_town.loc[university_town['RegionName'].str.endswith('[edit]')]
    university_town['State'] = university_town['State'].str.strip('[edit]').fillna(method='ffill')
    inv_map = dict(zip(states.values(), states.keys()))
    university_town['State'] = university_town['State'].map(inv_map)
    
    university_town = university_town[~university_town['RegionName'].str.contains('edit')]

    university_town['RegionName'] = university_town['RegionName'].str.split('(').str[0]
    university_town = university_town[['State','RegionName']]
    university_town['State'] = university_town['State'].str.strip()
    university_town['RegionName'] = university_town['RegionName'].str.strip()
    university_town = university_town.set_index(['State','RegionName'])
    
    return university_town

get_list_of_university_towns()

Empty DataFrame
Columns: []
Index: [(AL, Auburn), (AL, Florence), (AL, Jacksonville), (AL, Livingston), (AL, Montevallo), (AL, Troy), (AL, Tuscaloosa), (AL, Tuskegee), (AK, Fairbanks), (AZ, Flagstaff), (AZ, Tempe), (AZ, Tucson), (AR, Arkadelphia), (AR, Conway), (AR, Fayetteville), (AR, Jonesboro), (AR, Magnolia), (AR, Monticello), (AR, Russellville), (AR, Searcy), (CA, Angwin), (CA, Arcata), (CA, Berkeley), (CA, Chico), (CA, Claremont), (CA, Cotati), (CA, Davis), (CA, Irvine), (CA, Isla Vista), (CA, University Park, Los Angeles), (CA, Merced), (CA, Orange), (CA, Palo Alto), (CA, Pomona), (CA, Redlands), (CA, Riverside), (CA, Sacramento), (CA, University District, San Bernardino), (CA, San Diego), (CA, San Luis Obispo), (CA, Santa Barbara), (CA, Santa Cruz), (CA, Turlock), (CA, Westwood, Los Angeles), (CA, Whittier), (CO, Alamosa), (CO, Boulder), (CO, Durango), (CO, Fort Collins), (CO, Golden), (CO, Grand Junction), (CO, Greeley), (CO, Gunnison), (CO, Pueblo, Colorado), (nan, Fairfield), (nan, Middletown), (nan, New Britain), (nan, New Haven), (nan, New London), (nan, Storrs), (nan, Willimantic), (nan, Dover), (nan, Newark), (FL, Ave Maria), (FL, Boca Raton), (FL, Coral Gables), (FL, DeLand), (FL, Estero), (FL, Gainesville), (FL, Orlando), (FL, Sarasota), (FL, St. Augustine), (FL, St. Leo), (FL, Tallahassee), (FL, Tampa), (GA, Albany), (GA, Athens), (GA, Atlanta), (GA, Carrollton), (GA, Demorest), (GA, Fort Valley), (GA, Kennesaw), (GA, Milledgeville), (GA, Mount Vernon), (GA, Oxford), (GA, Rome), (GA, Savannah), (GA, Statesboro), (GA, Valdosta), (GA, Waleska), (GA, Young Harris), (nan, Manoa), (ID, Moscow), (ID, Pocatello), (ID, Rexburg), (IL, Carbondale), (IL, Champaign–Urbana), (IL, Charleston), (IL, DeKalb), (IL, Edwardsville), ...]

[517 rows x 0 columns]

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    import pandas as pd
    import numpy as np
    
    GDP = pd.read_excel('gdplev.xls', header=None, 
                        names=['Period', 'GDP in Billions of Current Dollars', 
                               'GDP in Billions of Chained 2009 Dollars'], usecols='E:G', skiprows=220)
    #--------------------------------Calculating GDP Growth and Decline----------------------------------------
    GDP['Difference'] = 0
    
    i = 0
    while i<(len(GDP)-1):
        GDP['Difference'].iloc[i] = GDP['GDP in Billions of Chained 2009 Dollars'].iloc[i] - GDP['GDP in Billions of Chained 2009 Dollars'].iloc[i+1]
        i+=1
        
    #------------------------Determining the Quarter at the Start of recession---------------------------------    
    i=0
    while i<(len(GDP)-1):
        if GDP['Difference'].iloc[i]>0 and GDP['Difference'].iloc[i+1]>0:
            x = (GDP['Period'].iloc[i+1])
            break
        i+=1
        
    return x

get_recession_start()

C:\Users\usa00\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    
    GDP = pd.read_excel('gdplev.xls', header=None, 
                        names=['Period', 'GDP in Billions of Current Dollars', 
                               'GDP in Billions of Chained 2009 Dollars'], usecols='E:G', skiprows=220)
    
    #--------------------------------Calculating GDP Growth and Decline----------------------------------------
    GDP['Difference'] = 0
    
    i = 0
    while i<(len(GDP)-1):
        GDP['Difference'].iloc[i] = GDP['GDP in Billions of Chained 2009 Dollars'].iloc[i] - GDP['GDP in Billions of Chained 2009 Dollars'].iloc[i+1]
        i+=1
    
    #-------------------------Determining the Quarter at the End of Recession----------------------------------
    i=0
    while i<(len(GDP)-1):
        if GDP['Difference'].iloc[i]>0 and GDP['Difference'].iloc[i+1]>0 and GDP['Difference'].iloc[i+2]<0 and GDP['Difference'].iloc[i+3]<0:
            x = (GDP['Period'].iloc[i+3])
            break
        i+=1
    
    return x

get_recession_end()

'2009q3'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    
    GDP = pd.read_excel('gdplev.xls', header=None, 
                        names=['Period', 'GDP in Billions of Current Dollars', 
                               'GDP in Billions of Chained 2009 Dollars'], usecols='E:G', skiprows=220)
    
    #--------------------------------Calculating GDP Growth and Decline----------------------------------------
    GDP['Difference'] = 0
    
    i = 0
    while i<(len(GDP)-1):
        GDP['Difference'].iloc[i] = GDP['GDP in Billions of Chained 2009 Dollars'].iloc[i] - GDP['GDP in Billions of Chained 2009 Dollars'].iloc[i+1]
        i+=1
    
    #-----------------------------------Determining Recession Bottome-------------------------------------------   
    i=0
    while i<(len(GDP)-1):
        if GDP['Difference'].iloc[i]>0 and GDP['Difference'].iloc[i+1]>0 and GDP['Difference'].iloc[i+2]<0 and GDP['Difference'].iloc[i+3]<0:
            x = (GDP['Period'].iloc[i+2])
            break
        i+=1
    
    return x

get_recession_bottom()

'2009q2'

In [13]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    import numpy as np
    
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv').drop(columns=['RegionID','Metro','CountyName','SizeRank',
                                                                       '1996-04','1996-05','1996-06','1996-07','1996-08',
                                                                      '1996-09','1996-10','1996-11','1996-12',
                                                                      '1997-01','1997-02','1997-03','1997-04','1997-05',
                                                                      '1997-06','1997-07','1997-08','1997-09','1997-10',
                                                                      '1997-11','1997-12','1998-01','1998-02','1998-03',
                                                                      '1998-04','1998-05','1998-06','1998-07','1998-08',
                                                                      '1998-09','1998-10','1998-11','1998-12',
                                                                       '1999-01','1999-02','1999-03','1999-04','1999-05',
                                                                      '1999-06','1999-07','1999-08','1999-09','1999-10',
                                                                      '1999-11','1999-12'])
    
    housing_data['State'] = housing_data['State'].str.strip()
    housing_data['RegionName'] = housing_data['RegionName'].str.strip()
    housing_data = housing_data.sort_values(by=['State','RegionName'])
    housing_data = housing_data.set_index(['State','RegionName'])
    housing_data = housing_data.groupby(pd.PeriodIndex(housing_data.columns, freq='q'), axis=1).mean()

    return housing_data

convert_housing_data_to_quarters()

2000Q1         2000Q2         2000Q3  \
State RegionName                                                  
AK    Anchor Point            NaN            NaN            NaN   
      Anchorage     174633.333333  175266.666667  179566.666667   
      Fairbanks     163200.000000  165033.333333  169300.000000   
      Homer                   NaN            NaN            NaN   
      Juneau        192466.666667  194300.000000  195166.666667   
...                           ...            ...            ...   
WY    Burns         101533.333333  104566.666667  108366.666667   
      Casper         89233.333333   89600.000000   89733.333333   
      Cheyenne      116866.666667  120033.333333  121533.333333   
      Evansville    128033.333333  128766.666667  130833.333333   
      Pine Bluffs    93733.333333   95066.666667   94633.333333   

                           2000Q4         2001Q1         2001Q2  \
State RegionName                                                  
AK    Anchor Point            NaN            NaN            NaN   
      Anchorage     182833.333333  182766.666667  183933.333333   
      Fairbanks     172800.000000  164433.333333  157800.000000   
      Homer                   NaN            NaN            NaN   
      Juneau        194166.666667  197166.666667  199533.333333   
...                           ...            ...            ...   
WY    Burns         113000.000000  115833.333333  117200.000000   
      Casper         93166.666667   95500.000000   97633.333333   
      Cheyenne      123633.333333  125533.333333  126300.000000   
      Evansville    132066.666667  130566.666667  131433.333333   
      Pine Bluffs    98066.666667  103233.333333  104600.000000   

                           2001Q3         2001Q4         2002Q1  \
State RegionName                                                  
AK    Anchor Point            NaN            NaN            NaN   
      Anchorage     188566.666667  191866.666667  193966.666667   
      Fairbanks     158200.000000  154666.666667  152766.666667   
      Homer                   NaN            NaN            NaN   
      Juneau        202433.333333  206300.000000  209066.666667   
...                           ...            ...            ...   
WY    Burns         117800.000000  117633.333333  117333.333333   
      Casper         99433.333333  100633.333333  101733.333333   
      Cheyenne      126466.666667  128133.333333  128466.666667   
      Evansville    132400.000000  133466.666667  133300.000000   
      Pine Bluffs   106500.000000  104066.666667  102233.333333   

                           2002Q2  ...         2014Q2         2014Q3  \
State RegionName                   ...                                 
AK    Anchor Point            NaN  ...  157800.000000  158000.000000   
      Anchorage     196700.000000  ...  278200.000000  280766.666667   
      Fairbanks     154533.333333  ...  209233.333333  207100.000000   
      Homer                   NaN  ...  233033.333333  232900.000000   
      Juneau        209866.666667  ...  328266.666667  331833.333333   
...                           ...  ...            ...            ...   
WY    Burns         117233.333333  ...  168866.666667  161933.333333   
      Casper        101533.333333  ...  175766.666667  177300.000000   
      Cheyenne      129633.333333  ...  177466.666667  176733.333333   
      Evansville    131066.666667  ...  296733.333333  305666.666667   
      Pine Bluffs   103566.666667  ...  148666.666667  154366.666667   

                           2014Q4         2015Q1         2015Q2  \
State RegionName                                                  
AK    Anchor Point  155566.666667  154266.666667  154100.000000   
      Anchorage     281700.000000  284166.666667  287166.666667   
      Fairbanks     207000.000000  207766.666667  206466.666667   
      Homer         231066.666667  233500.000000  238100.000000   
      Juneau        333200.000000  335666.666667  337366.666667   
...  

In [22]:
def run_ttest():
    import pandas as pd
    import numpy as np
    
    university_towns = get_list_of_university_towns()
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    housing_data = convert_housing_data_to_quarters()
    
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    price_ratio = pd.DataFrame(housing_data[recession_start].div(housing_data[recession_bottom]), columns = ['PriceRatio'])
    price_ratio = price_ratio#.dropna()
    
    price_ratio_unitowns = pd.merge(price_ratio, university_towns, on=['State','RegionName'], how='inner')
    price_ratio_non_unitowns = pd.merge(price_ratio, university_towns, on=['State','RegionName'], how='outer')
    
    better = 'University Town' if price_ratio_unitowns['PriceRatio'].mean()<price_ratio_non_unitowns['PriceRatio'].mean() else 'Non University Towns'
    
    #-----------------------------------------Running t test---------------------------------------------------------
    t,p = ttest_ind(price_ratio_unitowns['PriceRatio'],price_ratio_non_unitowns['PriceRatio'], nan_policy='omit')
    
    different = True if p<0.01 else False 
    
    return (different,p,better)
run_ttest()

(False, 0.02751501301774009, 'University Town')